## Create table to postgresql database: olist_db

In [6]:
import pandas as pd
import psycopg2
from sqlalchemy import create_engine, text

username='postgres'
password = 'admin'
hostname = 'localhost'
port = 5432
database ='olist_db'

db_url = f'postgresql://{username}:{password}@{hostname}:{port}/{database}'

# Create engine
engine = create_engine(db_url)

In [15]:
# # Connect to PostgreSQL
# conn = psycopg2.connect(
#     host="localhost",
#     database="olist_db",
#     user="postgres",
#     password="admin",
#     port = 5432
# )

# # Create a cursor object
# cursor = conn.cursor()

# File path and table name
csv_file = "clean_data/cleaned_orders_reviews.csv"
table_name = "orders_reviews.tbl"


# Read CSV into DataFrame
clean_orders_reviews_df = pd.read_csv(csv_file)

# Upload DataFrame to PostgreSQL
clean_orders_reviews_df.to_sql(table_name, engine, if_exists='append', index=False)



C:\Users\T470\AppData\Local\Temp\ipykernel_22424\6652182.py:19: DtypeWarning: Columns (8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  clean_orders_reviews_df = pd.read_csv(csv_file)
C:\Users\T470\AppData\Local\Temp\ipykernel_22424\6652182.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  clean_orders_reviews_df.to_sql(table_name, conn, if_exists='append', index=False)


DatabaseError: Execution failed on sql '
        SELECT
            name
        FROM
            sqlite_master
        WHERE
            type IN ('table', 'view')
            AND name=?;
        ': syntax error at or near ";"
LINE 8:             AND name=?;
                              ^




# Define SQL statements
primary_key_sql = "ALTER TABLE order_reviews.tbl ADD PRIMARY KEY (review_id);"
foreign_key_sql = "ALTER TABLE order_reviews.tbl ADD CONSTRAINT fk_order_id FOREIGN KEY (order_id) REFERENCES orders_tbl(order_id);"

# Execute SQL statements
cursor.execute(primary_key_sql)
cursor.execute(foreign_key_sql)

# Commit the changes
conn.commit()


In [ ]:
# Close the cursor and connection
cursor.close()
conn.close()